In [1]:
import sys
sys.path.append('/home/miguelcosta/Desktop/PBL-HGO')
import pandas as pd
# import pandasql as ps
#import matplotlib.pyplot as plt
# Import argsort
from numpy import argsort
from src.db.connection import *
from src.modelling_pipelines.final_model_pipeline import *
from src.modelling_pipelines.modelling_functions.xgboost import *
alertP1=connection("credentials.txt")
data = pre_process(alertP1)

X = data.Texto
y = data.result # Target variable

# split X and y into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=16,shuffle=False)


Tables: [(bytearray(b'ConsultaUrgencia_doentespedidosconsultaNeurologia2012'),), (bytearray(b'consultaneurologia2012'),), (bytearray(b'consultaneurologia201216anon_true'),), (bytearray(b'hgo_data_032023'),)]
308


/home/miguelcosta/Desktop/PBL-HGO/src/db/connection.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  alertP1 = pd.read_sql("""SELECT * FROM consultaneurologia201216anon_true""",mydb)
/home/miguelcosta/Desktop/PBL-HGO/src/pre_processing/data_cleaning.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['PROVENIENCIA'][alertP1['PROVENIENCIA']=='']='unknown'
/home/miguelcosta/Desktop/PBL-HGO/src/pre_processing/data_cleaning.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [30]:
from sklearn.decomposition import PCA
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from torch.utils.tensorboard import SummaryWriter
import torch

def train_biobert_with_dimension_reduction(X_train, y_train, epochs=10, lr=1e-5, batch_size=64, n_components=64):
    # Load BioBERTpt model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained('pucpr/biobertpt-all')
    model = AutoModelForSequenceClassification.from_pretrained('pucpr/biobertpt-all', num_labels=2, output_attentions=True)

    # Tokenize and encode sequences in the training set
    tokens_train = tokenizer.batch_encode_plus(
        X_train.tolist(),
        max_length = 200,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=False
    )

    # Convert integer sequences to tensors
    train_seq = torch.tensor(tokens_train['input_ids'])
    train_mask = torch.tensor(tokens_train['attention_mask'])
    train_y = torch.tensor(y_train.tolist())

    # Create data loader for training data
    train_data = TensorDataset(train_seq, train_mask, train_y)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    # Specify loss function (Cross-Entropy)
    optimizer = AdamW(model.parameters(), lr=lr)

    # Train model
    writer = SummaryWriter()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for step,batch in enumerate(train_dataloader):
            batch = [r.to(device) for r in batch]
            sent_id, mask, labels = batch
            optimizer.zero_grad()

            # Forward pass
            outputs = model(sent_id, mask, labels=labels)  # Pass labels to the model
            loss = outputs.loss  # Get the loss directly from the model's outputs
            total_loss += loss.item()

            # Extract embeddings for dimensionality reduction
            with torch.no_grad():
                output = model.base_model(sent_id, mask).last_hidden_state
                output = output.mean(dim=1)  # Average over tokens in each sequence

                # Fit PCA on the embeddings
                U, S, V = torch.pca_lowrank(output, q=n_components)

                loss.backward()
                optimizer.step()

            # ... (same as before)

        # ... (same as before)

    writer.close()

    return model, V


In [31]:
model, V = train_biobert_with_dimension_reduction(X_train, y_train, epochs=1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pucpr/biobertpt-all and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/miguelcosta/.conda/envs/aequitas/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/home/miguelcosta/.conda/envs/aequitas/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is 

ValueError: q(=64) must be non-negative integer and not greater than min(m, n)=7

In [29]:
pca = PCA(n_components=32)
# Extract embeddings for dimensionality reduction
with torch.no_grad():
    output = model.base_model(sent_id, mask).last_hidden_state
    output = output.mean(dim=1)  # Average over tokens in each sequence
    # Reshape output to 2D (sequences, embedding_dim)
    output_2d = output.view(output.shape[0], -1)
    # Fit PCA on the embeddings
    pca.fit(output.cpu().numpy())



NameError: name 'sent_id' is not defined

In [27]:
model, pca = train_biobert_with_dimension_reduction(X_train, y_train, epochs=1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pucpr/biobertpt-all and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/miguelcosta/.conda/envs/aequitas/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/home/miguelcosta/.conda/envs/aequitas/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is 

ValueError: n_components=64 must be between 0 and min(n_samples, n_features)=7 with svd_solver='full'

In [ ]:

def predict(model, pca, X_test, batch_size=32):
    tokenizer = AutoTokenizer.from_pretrained('pucpr/biobertpt-all')
    
    tokens_test = tokenizer.batch_encode_plus(
        X_test.tolist(),
        max_length=200,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=False
    )

    test_seq = torch.tensor(tokens_test['input_ids'])
    test_mask = torch.tensor(tokens_test['attention_mask'])

    test_data = TensorDataset(test_seq, test_mask)
    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    predictions = []
    
    model.eval()
    
    for batch in test_dataloader:
        batch = [r.to(device) for r in batch]
        sent_id, mask = batch

        with torch.no_grad():
            output = model.base_model(sent_id, mask).last_hidden_state
            output = output.mean(dim=1)  # Average over tokens in each sequence

            reduced_embeddings_test = pca.transform(output.cpu().numpy())

            outputs = model(reduced_embeddings_test)
            logits = outputs.logits

            logits = logits.detach().cpu().numpy()

            predictions.append(logits)

    return predictions


## ---------

In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from torch.utils.tensorboard import SummaryWriter
from sklearn.decomposition import PCA  # Import PCA from scikit-learn

def train_biobert_with_dimension_reduction(X_train, y_train, epochs=10, lr=1e-5, batch_size=32, n_components=64):
    # Load BioBERTpt model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained('pucpr/biobertpt-all')
    model = AutoModelForSequenceClassification.from_pretrained('pucpr/biobertpt-all', num_labels=2, output_attentions=True)

    # Tokenize and encode sequences in the training set
    tokens_train = tokenizer.batch_encode_plus(
        X_train.tolist(),
        max_length=200,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=False
    )

    # Convert integer sequences to tensors
    train_seq = torch.tensor(tokens_train['input_ids'])
    train_mask = torch.tensor(tokens_train['attention_mask'])
    train_y = torch.tensor(y_train.tolist())

    # Create data loader for training data
    train_data = TensorDataset(train_seq, train_mask, train_y)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    # Specify loss function (Cross-Entropy)
    optimizer = AdamW(model.parameters(), lr=lr)

    # Initialize PCA
    pca = PCA(n_components=n_components)
    
    # Train model
    writer = SummaryWriter()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        reduced_embeddings = []  # Initialize a list to store reduced embeddings
        for step, batch in enumerate(train_dataloader):
            batch = [r.to(device) for r in batch]
            sent_id, mask, labels = batch
            optimizer.zero_grad()

            # Forward pass
            outputs = model(sent_id, mask, labels=labels)  # Pass labels to the model
            loss = outputs.loss  # Get the loss directly from the model's outputs
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

            # Calculate and log the average loss for the current step
            avg_train_loss = total_loss / (step + 1)  # Step starts from 0
            writer.add_scalar('Loss/train (step)', avg_train_loss, step + epoch * len(train_dataloader))

            # Extract embeddings for dimensionality reduction
            with torch.no_grad():
                output = model.base_model(sent_id, mask).last_hidden_state
                output = output.mean(dim=1)  # Average over tokens in each sequence

                # Fit PCA on the embeddings
                pca.fit(output.cpu().numpy())


        # Concatenate reduced embeddings into a single tensor
        reduced_embeddings = torch.cat(reduced_embeddings, dim=0)

        # Perform PCA dimensionality reduction on the embeddings
        pca = PCA(n_components=n_components)
        reduced_embeddings = pca.fit_transform(reduced_embeddings.cpu().numpy())

        # Metrics for TensorBoard
        avg_train_loss = total_loss / len(train_dataloader)
        writer.add_scalar('Loss/train (epoch)', avg_train_loss, epoch)

        for param_group in optimizer.param_groups:
            writer.add_scalar('Learning_Rate', param_group['lr'], epoch)

        # Log model parameters and gradients
        # for name, param in model.named_parameters():
        #     writer.add_histogram('Parameters/' + name, param.data.cpu().numpy(), epoch)
        #     if param.grad is not None:
        #         writer.add_histogram('Gradients/' + name, param.grad.data.cpu().numpy(), epoch)

    writer.close()

    # Return the trained model and reduced embeddings
    return model, reduced_embeddings


/home/miguelcosta/.conda/envs/aequitas/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-10 14:57:41.476866: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-10 14:57:41.515557: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-10 14:57:41.515596: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-10 14:57:41.515616: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plug

In [4]:
model, reduced_embeddings = train_biobert_with_dimension_reduction(X_train, y_train, epochs=1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pucpr/biobertpt-all and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/miguelcosta/.conda/envs/aequitas/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/home/miguelcosta/.conda/envs/aequitas/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is 

In [21]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [22]:
def predict(model, pca, X_test, batch_size=32):
    # Tokenize and encode sequences in the test set
    tokenizer = AutoTokenizer.from_pretrained('pucpr/biobertpt-all')
    tokens_test = tokenizer.batch_encode_plus(
        X_test.tolist(),
        max_length=200,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=False
    )

    # Convert integer sequences to tensors
    test_seq = torch.tensor(tokens_test['input_ids'])
    test_mask = torch.tensor(tokens_test['attention_mask'])

    # Create data loader for test data
    test_data = TensorDataset(test_seq, test_mask)
    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Initialize lists to store predictions and true labels
    predictions = []
    
    # Predict labels for batches in the test set
    model.eval()
    for batch in test_dataloader:
        batch = [r.to(device) for r in batch]
        sent_id, mask = batch

        with torch.no_grad():
            output = model.base_model(sent_id, mask).last_hidden_state
            output = output.mean(dim=1)  # Average over tokens in each sequence

            # Apply PCA transformation on the embeddings
            reduced_embeddings_test = pca.transform(output.cpu().numpy())

            outputs = model(reduced_embeddings_test)
            logits = outputs.logits

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()

        # Store predictions and true labels
        predictions.append(logits)

    return predictions


In [23]:
predictions = predict(model, X_test)

NameError: name 'pca' is not defined

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, trainer
import torch

# Load BioBERTpt model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('pucpr/biobertpt-all')
model = AutoModelForSequenceClassification.from_pretrained('pucpr/biobertpt-all', num_labels=2, output_attentions=True)

# Tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    X_train.tolist(),
    max_length = 200,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pucpr/biobertpt-all and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/miguelcosta/.conda/envs/aequitas/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [3]:
model_output_dir = "biobertpt-finetuned-classification"
print(model_output_dir)

# Start TensorBoard before training to monitor it in progress
%load_ext tensorboard
%tensorboard --logdir '{model_output_dir}'/runs

biobertpt-finetuned-classification


Launching TensorBoard...

In [10]:
args = TrainingArguments(
    # output_dir: directory where the model checkpoints will be saved.
    output_dir=model_output_dir,
    # evaluation_strategy (default "no"):
    # Possible values are:
    # "no": No evaluation is done during training.
    # "steps": Evaluation is done (and logged) every eval_steps.
    # "epoch": Evaluation is done at the end of each epoch.
    evaluation_strategy="steps",
    # eval_steps: Number of update steps between two evaluations if
    # evaluation_strategy="steps". Will default to the same value as
    # logging_steps if not set.
    eval_steps=50,
    # logging_strategy (default: "steps"): The logging strategy to adopt during
    # training (used to log training loss for example). Possible values are:
    # "no": No logging is done during training.
    # "epoch": Logging is done at the end of each epoch.
    # "steps": Logging is done every logging_steps.
    logging_strategy="steps",
    # logging_steps (default 500): Number of update steps between two logs if
    # logging_strategy="steps".
    logging_steps=50,
    # save_strategy (default "steps"):
    # The checkpoint save strategy to adopt during training. Possible values are:
    # "no": No save is done during training.
    # "epoch": Save is done at the end of each epoch.
    # "steps": Save is done every save_steps (default 500).
    save_strategy="steps",
    # save_steps (default: 500): Number of updates steps before two checkpoint
    # saves if save_strategy="steps".
    save_steps=200,
    # learning_rate (default 5e-5): The initial learning rate for AdamW optimizer.
    # Adam algorithm with weight decay fix as introduced in the paper
    # Decoupled Weight Decay Regularization.
    learning_rate=2e-5,
    # per_device_train_batch_size: The batch size per GPU/TPU core/CPU for training.
    per_device_train_batch_size=16,
    # per_device_eval_batch_size: The batch size per GPU/TPU core/CPU for evaluation.
    per_device_eval_batch_size=16,
    # num_train_epochs (default 3.0): Total number of training epochs to perform
    # (if not an integer, will perform the decimal part percents of the last epoch
    # before stopping training).
    num_train_epochs=1,
    # load_best_model_at_end (default False): Whether or not to load the best model
    # found during training at the end of training.
    load_best_model_at_end=True,
    # metric_for_best_model:
    # Use in conjunction with load_best_model_at_end to specify the metric to use
    # to compare two different models. Must be the name of a metric returned by
    # the evaluation with or without the prefix "eval_".
    metric_for_best_model="accuracy",
    # report_to:
    # The list of integrations to report the results and logs to. Supported
    # platforms are "azure_ml", "comet_ml", "mlflow", "tensorboard" and "wandb".
    # Use "all" to report to all integrations installed, "none" for no integrations.
    report_to="tensorboard"
)


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
# Function that returns an untrained model to be trained
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
                                                              num_labels=2)

# Function that will be called at the end of each evaluation phase on the whole
# arrays of predictions/labels to produce metrics.
def compute_metrics(eval_pred):
    # Predictions and labels are grouped in a namedtuple called EvalPrediction
    predictions, labels = eval_pred
    # Get the index with the highest prediction score (i.e. the predicted labels)
    predictions = np.argmax(predictions, axis=1)
    # Compare the predicted labels with the reference labels
    results =  metric.compute(predictions=predictions, references=labels)
    # results: a dictionary with string keys (the name of the metric) and float
    # values (i.e. the metric values)
    return results

# Since PyTorch does not provide a training loop, the 🤗 Transformers library
# provides a Trainer API that is optimized for 🤗 Transformers models, with a
# wide range of training options and with built-in features like logging,
# gradient accumulation, and mixed precision.
trainer = Trainer(
    # Function that returns the model to train. It's useful to use a function
    # instead of directly the model to make sure that we are always training
    # an untrained model from scratch.
    model_init=model_init,
    # The training arguments.
    args=args,
    # The training dataset.
    train_dataset=splitted_datasets_encoded["train"],
    # The evaluation dataset. We use a small subset of the validation set
    # composed of 150 samples to speed up computations...
    eval_dataset=splitted_datasets_encoded["test"].shuffle(42).select(range(150)),
    # Even though the training set and evaluation set are already tokenized, the
    # tokenizer is needed to pad the "input_ids" and "attention_mask" tensors
    # to the length managed by the model. It does so one batch at a time, to
    # use less memory as possible.
    tokenizer=tokenizer,
    # Function that will be called at the end of each evaluation phase on the whole
    # arrays of predictions/labels to produce metrics.
    compute_metrics=compute_metrics
)

# ... train the model!
trainer.train()